In [1]:
import cv2
import imutils
import os

os.makedirs('Фотки', exist_ok=True)
cap = cv2.VideoCapture(r'C:\Users\Пажилой биофизик\Downloads\Собрание в канале _General_-20210406_161337-Запись собрания.mp4')
prev_frame = None
region = None
n_slide = 1
c = []
#prev_frame = cv2.imread('3.jpg')
#prev_frame = imutils.resize(prev_frame, width=1200)
#prev_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
#prev_frame = cv2.GaussianBlur(prev_frame, (5,5), 0)

while True:
    success, frame = cap.read()

    if success:
        frame = imutils.resize(frame, width=1200)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (5,5), 0)
        
        if cv2.waitKey(1) == 32: #Space
            region = cv2.selectROI('video', frame)
            (x, y, w, h) = [i for i in region]
            cv2.imwrite(f'{n_slide}.png', frame[y:y+h, x:x+w,:])
            n_slide += 1

        
        if prev_frame is None:
            prev_frame = gray
            continue

        
        frameDelta = cv2.absdiff(prev_frame, gray)
        thresh = cv2.threshold(frameDelta, 100, 255, cv2.THRESH_BINARY)[1]
        thresh = cv2.dilate(thresh, None, iterations=2)
        
        cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        
        if region is not None:
            if len(cnts) != 0:
                for c in cnts:
                    (x_, y_, w_, h_) = cv2.boundingRect(c)
                    if x_ > x and x_ < x + w and y_ > y and y_ < y + h:
                        cv2.imwrite(f'{n_slide}.png', frame[y:y+h, x:x+w,:])
                        n_slide += 1
                        break

        
        prev_frame = gray ###Закомментить, если есть начальная картинка
        cnts = []
        cv2.imshow('video', frame)
        
        if cv2.waitKey(1) == 27: #ESC
            break
            
            

    else:
        break
        
cap.release()
cv2.destroyAllWindows()